In [1]:
# VAE architecture


import torch
from models.base import BaseVAE
from torch import nn
from torch.nn import functional as F
from models.types_ import *


class ResidualBlockEncoder(nn.Module):
    
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 stride: int,
                 **kwargs) -> None:
        super(ResidualBlockEncoder, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                              kernel_size= 3, stride= stride, padding  = 1)
        self.conv2 = nn.Conv2d(in_channels, out_channels,
                              kernel_size= 3, stride= stride, padding  = 1)
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.norm(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            temp = self.conv2(x)
        else: temp = 1*x
        x = self.relu(self.conv1(x))
        x = x + temp
        return x
    
    
class ResidualBlockDecoder(nn.Module):
    
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 stride: int,
                 **kwargs) -> None:
        super(ResidualBlockDecoder, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        
        self.conv1 = nn.ConvTranspose2d(in_channels, 
                                        out_channels,
                                        kernel_size= 3, 
                                        stride= stride, 
                                        padding  = 1,
                                        output_padding=stride-1)
        
        self.conv2 = nn.ConvTranspose2d(in_channels, 
                                        out_channels,
                                        kernel_size= 3, 
                                        stride= stride, 
                                        padding  = 1,
                                        output_padding=stride-1)
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.LeakyReLU()
        
    def forward(self, x):
        x = self.norm(x)
        if self.stride != 1 or self.in_channels != self.out_channels:
            temp = self.conv2(x)
        else: temp = 1*x
        x = self.relu(self.conv1(x))
        x = x + temp
        return x

        
class VAE(BaseVAE):

    def __init__(self,
                 in_channels: int,
                 in_size: int, 
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super(VAE, self).__init__()

        self.latent_dim = latent_dim
        self.nb_last_channels = hidden_dims[-1]
        self.out_channels = in_channels
        

        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]
            div = 2**5  # Model reduces size by this factor           
        else:
            div = 2**len(hidden_dims)   # Model reduces size by this factor   
            
        # Make sure input dimension is usable     
        if in_size%div==0:
            self.smallest_size = int(in_size/div)
        else: raise ValueError('Input size not compatible with number of hidden layers.')

            

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=in_channels, stride= 1),
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=in_channels, stride= 1),
                    ResidualBlockEncoder(in_channels=in_channels, out_channels=h_dim, stride= 2))
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*self.smallest_size**2, latent_dim)


        # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * self.smallest_size**2)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i], stride = 1),
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i], stride = 1),
                    ResidualBlockDecoder(hidden_dims[i], hidden_dims[i+1], stride = 2))
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= self.out_channels,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

    def encode(self, input: Tensor) -> List[Tensor]:
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        result = self.decoder_input(z)
        result = result.view(-1, self.nb_last_channels, self.smallest_size, self.smallest_size)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)


        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':-kld_loss}

    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

In [2]:
# Define Trainer

from torchvision.utils import save_image

# Achtung: Momentan wird im Trainer nur Reconstruction Loss verwendet
class Trainer(object):
    def __init__(self, model,
                 optimizer, loss_function,
                 loader_train, loader_val,
                 dtype, device, **in_params):
        """
        :param model: PyTorch model of the neural network

        :param optimizer: PyTorch optimizer

        :param print_every: How often should we print the loss during training
        """
        # Create attributes:
        self.device = device
        self.model = model.to(device=self.device)  # move the model parameters to CPU/GPU
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.loader_train = loader_train
        self.loader_val = loader_val
        self.print_every = in_params["print_every"]
        self.dtype = dtype
        self.batch_size = in_params["batch_size"]
        self.input_size = in_params["input_size"]
        self.path = in_params["path"]


    def train_model(self, epoch):
        """
        - epoch: An integer giving the epoch
        """
        train_loss = 0
        self.model.train()  # put model to training mode
        for t, (input,_) in enumerate(self.loader_train):
            
            input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
            
            # do a step in training
            args = self.model(input)
            loss = self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_train)})['Reconstruction_Loss']
            self.optimizer.zero_grad()
            loss.backward() 
            torch.nn.utils.clip_grad_norm_(self.model.parameters(),1)
            train_loss += loss.item() # accumulate for average loss
            self.optimizer.step()

            # print loss
            if t % self.print_every == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, t * len(args[1]), len(self.loader_train.dataset),
                    100. * t / len(self.loader_train),
                    loss.item() / len(args[1])))
        # print average loss
        print('====> Epoch: {} Average loss: {:.6f}'.format(
              epoch, train_loss / len(self.loader_train.dataset)))

    def test_model(self, epoch):
        self.model.eval() # Put model to evaluation mode
        test_loss = 0.

        with torch.no_grad():
            # During validation, we accumulate these values across the whole dataset and then average at the end:
            for i, (input,_) in enumerate(self.loader_val):
                input = input.to(device=self.device, dtype=self.dtype)  # move to device, e.g. GPU
       
                # compute loss and accumulate
                args = self.model(input)
                test_loss += self.loss_function(*args,**{'M_N':self.batch_size/len(self.loader_val)})['Reconstruction_Loss'].item()
                if i == 0:
                    n = min(args[1].size(0), 8)
                    comparison = torch.cat([args[1][:n],
                                          args[0].view(self.batch_size, self.model.out_channels, self.input_size, self.input_size)[:n]])
                    save_image(comparison.cpu(),
                             self.path + '/reconstruction_' + str(epoch) + '.png', nrow=n)

        # print average loss
        test_loss /= len(self.loader_val.dataset)
        print('====> Test set loss: {:.6f}'.format(test_loss))
        
    def train_and_test(self, epochs):
        for e in range(1,epochs+1):
            self.train_model(e)
            self.test_model(e)
            with torch.no_grad():
                sample = self.model.sample(64,device)
                save_image(sample.view(64, self.model.out_channels, self.input_size, self.input_size),
                           self.path + '/sample_' + str(e) + '.png')

In [ ]:
# Train VAE

from torchvision import datasets, transforms

in_params = {"batch_size": 32,
        "epochs": 2000,
        "no_cuda": False,
        "seed": 1,
        "print_every": 10,
        "input_size": 64,
        "path": 'results_trial'
        }
in_params["cuda"] = not in_params["no_cuda"] and torch.cuda.is_available()
torch.manual_seed(in_params["seed"])

device = torch.device("cuda" if in_params["cuda"] else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if in_params["cuda"] else {}


model = VAE(in_channels=3,
           in_size=in_params["input_size"],
           latent_dim=250,
           hidden_dims=[32, 64, 128, 160])

# transformations of input images before feeding into nn
transformations = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomCrop(in_params["input_size"]),
    transforms.ColorJitter(0.4,0.4,0.4,0.0),
    transforms.ToTensor()])

#load data into DataLoader
train_dataset = datasets.ImageFolder('data_to_try/full_data/training', 
                                 transform=transformations
                                   )
test_dataset = datasets.ImageFolder('data_to_try/full_data/test', 
                                 transform=transformations
                                   )



train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=in_params["batch_size"], 
                                           shuffle=True,
                                           drop_last=True,
                                           **kwargs
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=in_params["batch_size"], 
                                          shuffle=True,
                                          drop_last=True,
                                          **kwargs
                                          )

# Build the optimizer:
params = model.parameters()
learning_rate = 1e-4
optimizer = torch.optim.AdamW(params, lr=learning_rate)


# Build the trainer with the Soresen-Dice loss you implemented:
trainer = Trainer(model, optimizer, model.loss_function,
        train_loader, test_loader, torch.float32, device,**in_params )

# Start training:
trainer.train_and_test(in_params["epochs"])

Train Epoch: 1 [0/1620 (0%)]	Loss: 0.005063
Train Epoch: 1 [320/1620 (20%)]	Loss: 0.004374
Train Epoch: 1 [640/1620 (40%)]	Loss: 0.002238
Train Epoch: 1 [960/1620 (60%)]	Loss: 0.002202
Train Epoch: 1 [1280/1620 (80%)]	Loss: 0.001417
====> Epoch: 1 Average loss: 0.002859
====> Test set loss: 0.001370
Train Epoch: 2 [0/1620 (0%)]	Loss: 0.001300
Train Epoch: 2 [320/1620 (20%)]	Loss: 0.000653
Train Epoch: 2 [640/1620 (40%)]	Loss: 0.001072
Train Epoch: 2 [960/1620 (60%)]	Loss: 0.000577
Train Epoch: 2 [1280/1620 (80%)]	Loss: 0.000709
====> Epoch: 2 Average loss: 0.000854
====> Test set loss: 0.000883
Train Epoch: 3 [0/1620 (0%)]	Loss: 0.000484
Train Epoch: 3 [320/1620 (20%)]	Loss: 0.000573
Train Epoch: 3 [640/1620 (40%)]	Loss: 0.000654
Train Epoch: 3 [960/1620 (60%)]	Loss: 0.000667
Train Epoch: 3 [1280/1620 (80%)]	Loss: 0.000721
====> Epoch: 3 Average loss: 0.000601
====> Test set loss: 0.000691
Train Epoch: 4 [0/1620 (0%)]	Loss: 0.000646
Train Epoch: 4 [320/1620 (20%)]	Loss: 0.000635
Train 

====> Test set loss: 0.000378
Train Epoch: 28 [0/1620 (0%)]	Loss: 0.000276
Train Epoch: 28 [320/1620 (20%)]	Loss: 0.000231
Train Epoch: 28 [640/1620 (40%)]	Loss: 0.000265
Train Epoch: 28 [960/1620 (60%)]	Loss: 0.000294
Train Epoch: 28 [1280/1620 (80%)]	Loss: 0.000237
====> Epoch: 28 Average loss: 0.000256
====> Test set loss: 0.000394
Train Epoch: 29 [0/1620 (0%)]	Loss: 0.000199
Train Epoch: 29 [320/1620 (20%)]	Loss: 0.000230
Train Epoch: 29 [640/1620 (40%)]	Loss: 0.000204
Train Epoch: 29 [960/1620 (60%)]	Loss: 0.000240
Train Epoch: 29 [1280/1620 (80%)]	Loss: 0.000236
====> Epoch: 29 Average loss: 0.000247
====> Test set loss: 0.000394
Train Epoch: 30 [0/1620 (0%)]	Loss: 0.000233
Train Epoch: 30 [320/1620 (20%)]	Loss: 0.000243
Train Epoch: 30 [640/1620 (40%)]	Loss: 0.000254
Train Epoch: 30 [960/1620 (60%)]	Loss: 0.000230
Train Epoch: 30 [1280/1620 (80%)]	Loss: 0.000291
====> Epoch: 30 Average loss: 0.000244
====> Test set loss: 0.000340
Train Epoch: 31 [0/1620 (0%)]	Loss: 0.000211
Trai

Train Epoch: 54 [1280/1620 (80%)]	Loss: 0.000237
====> Epoch: 54 Average loss: 0.000214
====> Test set loss: 0.000311
Train Epoch: 55 [0/1620 (0%)]	Loss: 0.000207
Train Epoch: 55 [320/1620 (20%)]	Loss: 0.000207
Train Epoch: 55 [640/1620 (40%)]	Loss: 0.000172
Train Epoch: 55 [960/1620 (60%)]	Loss: 0.000186
Train Epoch: 55 [1280/1620 (80%)]	Loss: 0.000176
====> Epoch: 55 Average loss: 0.000201
====> Test set loss: 0.000296
Train Epoch: 56 [0/1620 (0%)]	Loss: 0.000263
Train Epoch: 56 [320/1620 (20%)]	Loss: 0.000250
Train Epoch: 56 [640/1620 (40%)]	Loss: 0.000205
Train Epoch: 56 [960/1620 (60%)]	Loss: 0.000248
Train Epoch: 56 [1280/1620 (80%)]	Loss: 0.000172
====> Epoch: 56 Average loss: 0.000204
====> Test set loss: 0.000306
Train Epoch: 57 [0/1620 (0%)]	Loss: 0.000223
Train Epoch: 57 [320/1620 (20%)]	Loss: 0.000185
Train Epoch: 57 [640/1620 (40%)]	Loss: 0.000199
Train Epoch: 57 [960/1620 (60%)]	Loss: 0.000157
Train Epoch: 57 [1280/1620 (80%)]	Loss: 0.000201
====> Epoch: 57 Average loss: 

Train Epoch: 81 [640/1620 (40%)]	Loss: 0.000200
Train Epoch: 81 [960/1620 (60%)]	Loss: 0.000182
Train Epoch: 81 [1280/1620 (80%)]	Loss: 0.000172
====> Epoch: 81 Average loss: 0.000174
====> Test set loss: 0.000259
Train Epoch: 82 [0/1620 (0%)]	Loss: 0.000190
Train Epoch: 82 [320/1620 (20%)]	Loss: 0.000168
Train Epoch: 82 [640/1620 (40%)]	Loss: 0.000148
Train Epoch: 82 [960/1620 (60%)]	Loss: 0.000308
Train Epoch: 82 [1280/1620 (80%)]	Loss: 0.000211
====> Epoch: 82 Average loss: 0.000175
====> Test set loss: 0.000282
Train Epoch: 83 [0/1620 (0%)]	Loss: 0.000166
Train Epoch: 83 [320/1620 (20%)]	Loss: 0.000205
Train Epoch: 83 [640/1620 (40%)]	Loss: 0.000130
Train Epoch: 83 [960/1620 (60%)]	Loss: 0.000155
Train Epoch: 83 [1280/1620 (80%)]	Loss: 0.000138
====> Epoch: 83 Average loss: 0.000163
====> Test set loss: 0.000287
Train Epoch: 84 [0/1620 (0%)]	Loss: 0.000171
Train Epoch: 84 [320/1620 (20%)]	Loss: 0.000143
Train Epoch: 84 [640/1620 (40%)]	Loss: 0.000288
Train Epoch: 84 [960/1620 (60%)

====> Test set loss: 0.000255
Train Epoch: 108 [0/1620 (0%)]	Loss: 0.000142
Train Epoch: 108 [320/1620 (20%)]	Loss: 0.000116
Train Epoch: 108 [640/1620 (40%)]	Loss: 0.000167
Train Epoch: 108 [960/1620 (60%)]	Loss: 0.000172
Train Epoch: 108 [1280/1620 (80%)]	Loss: 0.000110
====> Epoch: 108 Average loss: 0.000149
====> Test set loss: 0.000258
Train Epoch: 109 [0/1620 (0%)]	Loss: 0.000154
Train Epoch: 109 [320/1620 (20%)]	Loss: 0.000137
Train Epoch: 109 [640/1620 (40%)]	Loss: 0.000152
Train Epoch: 109 [960/1620 (60%)]	Loss: 0.000168
Train Epoch: 109 [1280/1620 (80%)]	Loss: 0.000147
====> Epoch: 109 Average loss: 0.000147
====> Test set loss: 0.000263
Train Epoch: 110 [0/1620 (0%)]	Loss: 0.000135
Train Epoch: 110 [320/1620 (20%)]	Loss: 0.000139
Train Epoch: 110 [640/1620 (40%)]	Loss: 0.000195
Train Epoch: 110 [960/1620 (60%)]	Loss: 0.000121
Train Epoch: 110 [1280/1620 (80%)]	Loss: 0.000143
====> Epoch: 110 Average loss: 0.000149
====> Test set loss: 0.000269
Train Epoch: 111 [0/1620 (0%)]	

...

====> Epoch: 370 Average loss: 0.000094
====> Test set loss: 0.000231
Train Epoch: 371 [0/1620 (0%)]	Loss: 0.000091
Train Epoch: 371 [320/1620 (20%)]	Loss: 0.000073
Train Epoch: 371 [640/1620 (40%)]	Loss: 0.000092
Train Epoch: 371 [960/1620 (60%)]	Loss: 0.000097
Train Epoch: 371 [1280/1620 (80%)]	Loss: 0.000096
====> Epoch: 371 Average loss: 0.000099
====> Test set loss: 0.000247
Train Epoch: 372 [0/1620 (0%)]	Loss: 0.000087
Train Epoch: 372 [320/1620 (20%)]	Loss: 0.000084
Train Epoch: 372 [640/1620 (40%)]	Loss: 0.000104
Train Epoch: 372 [960/1620 (60%)]	Loss: 0.000087
Train Epoch: 372 [1280/1620 (80%)]	Loss: 0.000102
====> Epoch: 372 Average loss: 0.000096
====> Test set loss: 0.000238
Train Epoch: 373 [0/1620 (0%)]	Loss: 0.000083
Train Epoch: 373 [320/1620 (20%)]	Loss: 0.000089
Train Epoch: 373 [640/1620 (40%)]	Loss: 0.000098
Train Epoch: 373 [960/1620 (60%)]	Loss: 0.000095
Train Epoch: 373 [1280/1620 (80%)]	Loss: 0.000132
====> Epoch: 373 Average loss: 0.000096
====> Test set loss: 

Train Epoch: 397 [0/1620 (0%)]	Loss: 0.000099
Train Epoch: 397 [320/1620 (20%)]	Loss: 0.000114
Train Epoch: 397 [640/1620 (40%)]	Loss: 0.000134
Train Epoch: 397 [960/1620 (60%)]	Loss: 0.000122
Train Epoch: 397 [1280/1620 (80%)]	Loss: 0.000117
====> Epoch: 397 Average loss: 0.000102
====> Test set loss: 0.000237
Train Epoch: 398 [0/1620 (0%)]	Loss: 0.000139
Train Epoch: 398 [320/1620 (20%)]	Loss: 0.000097
Train Epoch: 398 [640/1620 (40%)]	Loss: 0.000097
Train Epoch: 398 [960/1620 (60%)]	Loss: 0.000104
Train Epoch: 398 [1280/1620 (80%)]	Loss: 0.000078
====> Epoch: 398 Average loss: 0.000095
====> Test set loss: 0.000230
Train Epoch: 399 [0/1620 (0%)]	Loss: 0.000077
Train Epoch: 399 [320/1620 (20%)]	Loss: 0.000085
Train Epoch: 399 [640/1620 (40%)]	Loss: 0.000089
Train Epoch: 399 [960/1620 (60%)]	Loss: 0.000106
Train Epoch: 399 [1280/1620 (80%)]	Loss: 0.000108
====> Epoch: 399 Average loss: 0.000095
====> Test set loss: 0.000211
Train Epoch: 400 [0/1620 (0%)]	Loss: 0.000092
Train Epoch: 40

Train Epoch: 423 [640/1620 (40%)]	Loss: 0.000096
Train Epoch: 423 [960/1620 (60%)]	Loss: 0.000146
Train Epoch: 423 [1280/1620 (80%)]	Loss: 0.000142
====> Epoch: 423 Average loss: 0.000092
====> Test set loss: 0.000229
Train Epoch: 424 [0/1620 (0%)]	Loss: 0.000107
Train Epoch: 424 [320/1620 (20%)]	Loss: 0.000067
Train Epoch: 424 [640/1620 (40%)]	Loss: 0.000076
Train Epoch: 424 [960/1620 (60%)]	Loss: 0.000079
Train Epoch: 424 [1280/1620 (80%)]	Loss: 0.000071
====> Epoch: 424 Average loss: 0.000089
====> Test set loss: 0.000240
Train Epoch: 425 [0/1620 (0%)]	Loss: 0.000096
Train Epoch: 425 [320/1620 (20%)]	Loss: 0.000067
Train Epoch: 425 [640/1620 (40%)]	Loss: 0.000097
Train Epoch: 425 [960/1620 (60%)]	Loss: 0.000078
Train Epoch: 425 [1280/1620 (80%)]	Loss: 0.000123
====> Epoch: 425 Average loss: 0.000088
====> Test set loss: 0.000237
Train Epoch: 426 [0/1620 (0%)]	Loss: 0.000109
Train Epoch: 426 [320/1620 (20%)]	Loss: 0.000104
Train Epoch: 426 [640/1620 (40%)]	Loss: 0.000089
Train Epoch:

Train Epoch: 449 [1280/1620 (80%)]	Loss: 0.000097
====> Epoch: 449 Average loss: 0.000086
====> Test set loss: 0.000207
Train Epoch: 450 [0/1620 (0%)]	Loss: 0.000072
Train Epoch: 450 [320/1620 (20%)]	Loss: 0.000101
Train Epoch: 450 [640/1620 (40%)]	Loss: 0.000095
Train Epoch: 450 [960/1620 (60%)]	Loss: 0.000072
Train Epoch: 450 [1280/1620 (80%)]	Loss: 0.000088
====> Epoch: 450 Average loss: 0.000086
====> Test set loss: 0.000232
Train Epoch: 451 [0/1620 (0%)]	Loss: 0.000133
Train Epoch: 451 [320/1620 (20%)]	Loss: 0.000117
Train Epoch: 451 [640/1620 (40%)]	Loss: 0.000080
Train Epoch: 451 [960/1620 (60%)]	Loss: 0.000082
Train Epoch: 451 [1280/1620 (80%)]	Loss: 0.000089
====> Epoch: 451 Average loss: 0.000090
====> Test set loss: 0.000205
Train Epoch: 452 [0/1620 (0%)]	Loss: 0.000097
Train Epoch: 452 [320/1620 (20%)]	Loss: 0.000111
Train Epoch: 452 [640/1620 (40%)]	Loss: 0.000086
Train Epoch: 452 [960/1620 (60%)]	Loss: 0.000074
Train Epoch: 452 [1280/1620 (80%)]	Loss: 0.000081
====> Epoch

====> Test set loss: 0.000220
Train Epoch: 476 [0/1620 (0%)]	Loss: 0.000096
Train Epoch: 476 [320/1620 (20%)]	Loss: 0.000102
Train Epoch: 476 [640/1620 (40%)]	Loss: 0.000112
Train Epoch: 476 [960/1620 (60%)]	Loss: 0.000105
Train Epoch: 476 [1280/1620 (80%)]	Loss: 0.000090
====> Epoch: 476 Average loss: 0.000090
====> Test set loss: 0.000228
Train Epoch: 477 [0/1620 (0%)]	Loss: 0.000088
Train Epoch: 477 [320/1620 (20%)]	Loss: 0.000085
Train Epoch: 477 [640/1620 (40%)]	Loss: 0.000074
Train Epoch: 477 [960/1620 (60%)]	Loss: 0.000096
Train Epoch: 477 [1280/1620 (80%)]	Loss: 0.000092
====> Epoch: 477 Average loss: 0.000090
====> Test set loss: 0.000226
Train Epoch: 478 [0/1620 (0%)]	Loss: 0.000090
Train Epoch: 478 [320/1620 (20%)]	Loss: 0.000103
Train Epoch: 478 [640/1620 (40%)]	Loss: 0.000073
Train Epoch: 478 [960/1620 (60%)]	Loss: 0.000085
Train Epoch: 478 [1280/1620 (80%)]	Loss: 0.000087
====> Epoch: 478 Average loss: 0.000090
====> Test set loss: 0.000216
Train Epoch: 479 [0/1620 (0%)]	